In [ ]:
import sys

sys.path.append("../")
from helper.utils import *

from helper.catboost import *
from config import *


In [ ]:
is_debug = False
nrows = 1000

In [ ]:
version = "0"

In [ ]:
if is_debug:
    train = pd.read_csv(
        join(proc_data_path, "train_{}.csv".format(version)), nrows=nrows
    )
    test = pd.read_csv(
        join(proc_data_path, "test_{}.csv".format(version)), nrows=nrows / 2
    )
else:
    train = pd.read_csv(join(proc_data_path, "train_{}.csv".format(version)))
    test = pd.read_csv(join(proc_data_path, "test_{}.csv".format(version)))

In [ ]:
train.columns

In [ ]:
target_name = "target"
Id_name = "ward"
prediction_names = ["{}_prediction".format(target_name)]

features_to_remove = [
    target_name,
    Id_name,
    "train",
    "validation",
    "fold",
    #     "area_4",
    "area_3",
    "area_2",
    "area_1",
    "distance_to_ADM1",
    "distance_to_ADM2",
    "distance_to_ADM3",
] + prediction_names
features = [
    feature for feature in train.columns.tolist() if feature not in features_to_remove
]
data_characterization(train[features])

In [ ]:
print(features)

In [ ]:
from sklearn.metrics import mean_squared_error


def metric(x, y):
    return np.sqrt(mean_squared_error(x, y))

[catboost params documentation](https://catboost.ai/docs/concepts/python-reference_parameters-list.html)

In [ ]:
params = {
    "loss_function": "RMSE",
    "eval_metric": "RMSE",
    "learning_rate": 0.05,
    "random_seed": 1994,
    "l2_leaf_reg": 3,
    "bagging_temperature": 1,  # 0 inf

    "depth": 4,
    #         "min_data_in_leaf":1,
    #         "max_leaves":31,
    #     "subsample": 0.8,
    #         "sampling_frequency":"PerTreeLevel", # PerTree
    "od_type": "Iter",
    "od_wait": 50,
    "thread_count": 16,
    "iterations": 50000,
    "verbose_eval": False,
    "use_best_model": True,
}

In [ ]:
cat_features = []
cat_features = [
    train[features].columns.get_loc(c) for c in cat_features if c in train[features]
]

In [ ]:
other_params = {
    "prediction_type": "RawFormulaVal",  # it could be RawFormulaVal ,Class,Probability
    "cat_features": cat_features,
    "print_result": False,  # print result for a single model should be False whene use_kfold==True
    "plot_importance": False,  # plot importance for single model should be false whene use_kfold==True
    "predict_train": False,  # predict train for the single model funcation False only whene  use_kfold==True
    "num_class": 1,
    "target_name": target_name,
    "features": features,
    "metric": metric,
    "params": params,
    "use_kfold": True,  # condtion to use kfold or single model
    "plot_importance_kfold": False,  # plot importance after K fold train
    "print_kfold_eval": False,  # print evalation in kfold mode
    "weight": None,
    "print_time": False,
}

In [ ]:
from tqdm import tqdm

In [ ]:
oof_validations = []
oof_tests = []
for seed in tqdm(
    [
        1994,
        2021,
        2040,
        45678,
        45210,
        48796,
        521426,
        74582,
        457895,
        458763,
        52478,
        78524,
        32105,
    ]
):


    other_params["params"]["random_seed"] = seed
    if other_params["use_kfold"]:
        oof_train, oof_test, final_train_score, oof_score, models = cat_train(
            train, test, other_params
        )
        validation = fill_predictions_df(train, oof_train, prediction_names)

    else:
        train_pred, val_pred, test_pred, train_score, val_score, model = cat_train(
            train, test, other_params
        )
        validation = fill_predictions_df(
            train[train.validation == 1], val_pred, prediction_names
        )
    oof_validations.append(oof_train)
    oof_tests.append(oof_test)

In [ ]:
validation = fill_predictions_df(
    train, np.mean(oof_validations, axis=0), prediction_names
)
oof_test = np.mean(oof_tests, axis=0)


In [ ]:
score = metric(validation[target_name], validation[prediction_names].values)
print(score)
print("Train")
print("train mean ", train[target_name].mean(), "train std ", train[target_name].std())
print(
    "oof mean ",
    np.mean(validation[prediction_names].values),
    "oof sutd ",
    np.std(validation[prediction_names].values),
)

In [ ]:
sub_name = "cat_{}_{}".format(round(score, 3), str("Kfold"))
make_sub_one_class(test, oof_test, Id_name, target_name, join(sub_path, sub_name))

In [ ]:
sns.kdeplot(validation[prediction_names].values.reshape((-1)))
sns.kdeplot(validation[target_name].values.reshape((-1)))
sns.kdeplot(test["oof_test"])